In [ ]:
'''
pip install pyspark

PYSPARK_DRIVER_PYTHON = python

!pip install findspark

import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark_session = SparkSession.builder.appName('cluster').getOrCreate()
spark_session


spark_session.stop()
'''

In [ ]:
#!pip install findspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import *


In [ ]:
import findspark
findspark.init()
findspark.find()

In [ ]:
Schema = StructType([
    StructField("numpoliza", StringType(), True),
    StructField("numorden", LongType(), True),
    StructField("rutbeneficiario", LongType(), True),
    StructField("dvrutbeneficiario", StringType(), True),
    StructField("apellidopaterno", StringType(), True),
    StructField("apellidomaterno", StringType(), True),
    StructField("nombres", StringType(), True),
    StructField("sexo", StringType(), True),
    StructField("tipobeneficiario", StringType(), True),
    StructField("situacioninvalidez", StringType(), True),
    StructField("fechanacimiento", StringType(), True),
    StructField("edad", LongType(), True),
    StructField("numdependentes", LongType(), True),
    StructField("fechafallecimiento", StringType(), True),
    StructField("fechainvalidez", StringType(), True),
    StructField("derechopension", StringType(), True),
    StructField("requisitopension", StringType(), True),
    StructField("relacionhijomadre", StringType(), True),
    StructField("fechanacimientohijomenor", StringType(), True),
    StructField("derechoacrecer", StringType(), True),
    StructField("porcentajepension", LongType(), True),
    StructField("pensionpersona", StringType(), True)]
)
    

In [ ]:
#sc = SparkContext(appname="name")
spark_session = SparkSession.builder.appName('cluster').getOrCreate()
spark_session

In [ ]:
dataset = spark_session.read.option("header",True).schema(schema=Schema).csv("tb_Previsionales_Beneficiarios.csv")

In [ ]:
dataset.printSchema()

In [ ]:
dataset.count()

In [ ]:
#remove NA from all columns
#dataset = dataset.dropna('any')
#dataset.count()

In [ ]:
#distinct count edad
#dataset.select("edad").dropDuplicates().count()
dataset.select("tipobeneficiario").distinct().collect()

In [ ]:
#fill na
dataset.fillna(0, subset=["tipobeneficiario"]).count()

In [ ]:
#replace null by empty

dataset.na.fill("", subset=["fechanacimientohijomenor"])

In [ ]:
#Where
dataset.where("tipobeneficiario='50'").count()

In [ ]:
#replace columns values
from pyspark.sql.functions import when

dataset = dataset.withColumn("tipobeneficiario",
        when(dataset["tipobeneficiario"]=="50","999").otherwise(dataset["tipobeneficiario"]))

In [ ]:
#Cast variables
dataset = dataset.withColumn("edad", dataset["edad"].cast("Integer"))
#dataset.printSchema()


In [ ]:
#GroupBy
dataset.groupBy("sexo").mean("edad").show()

In [ ]:
#orderby

dataset.orderBy("nombres").show(truncate = True)

In [ ]:
dataset.createOrReplaceTempView("datasetSQL")

#spark_session.sql("Select distinct dvrutbeneficiario from datasetSQL").show()
spark_session.sql("SELECT count(tipobeneficiario) FROM datasetSQL WHERE tipobeneficiario != ''").show()
                  

In [ ]:
dataset = dataset.withColumn("newcolumn", dataset['column'].split(col("column"),"/").getitem(0))

In [ ]:
dataset.createOrReplaceTempView("new_table")
spark_session.sql("Select * from new_table")

In [ ]:
spark_session.stop()
